## 用决策树来分类贷款是否优良

[LendingClub](https://www.lendingclub.com/) 是一家贷款公司. 在本次作业中,我们需要手动实现决策树来预测一份贷款是否安全，并对比不同复杂度下决策树的表现

In [ ]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import cross_val_score, train_test_split
from sklearn import tree
from sklearn import metrics




## 读取数据

In [ ]:
#data = pd.read_csv(os.path.join("data", "loan_sub.csv"), sep=',')
data = pd.read_csv(os.path.join("../input", "loan_sub.csv"), sep=',')

## 打印可用特征

In [ ]:
data.bad_loans.dtype

## 预处理预测目标

预测目标是一列'bad_loans'的数据。其中**1**表示的是不良贷款，**0**表示的是优质贷款。

将预测目标处理成更符合直觉的标签，创建一列 `safe_loans`. 并且: 

* **+1** 表示优质贷款, 
* **-1** 表示不良贷款. 

In [ ]:
# safe_loans =  1 => safe
# safe_loans = -1 => risky
#TODO
df1 = pd.DataFrame({'safe_loans' : data.bad_loans.apply(lambda x : +1 if x==0 else -1)})
data = pd.concat([data, df1], axis = 1)
data = data.drop('bad_loans', axis = 1)

## 打印优质贷款与不良贷款的比例

In [ ]:
print(data.head())
data.drop_duplicates()
data['safe_loans'].value_counts(normalize=True)

#### 这是一个不平衡数据， 好的贷款远比坏的贷款要多. 

## 选取用于预测的特征

In [ ]:
cols = ['grade', 'term','home_ownership', 'emp_length']
target = 'safe_loans'

data = data[cols + [target]]#TODO
data.head()

## 创建更为平衡的数据集  

* 对占多数的标签进行下采样  
* 注意有很多方法处理不平衡数据，下采样只是其中之一

In [ ]:
data['safe_loans'].value_counts()


In [ ]:

# use the percentage of bad and good loans to undersample the safe loans.
bad_ones = data[data[target] == -1]
safe_ones = data[data[target] == 1]
percentage = len(bad_ones)/float(len(safe_ones))#TODO

risky_loans = bad_ones
# Important, this step is to balance out the sample
safe_loans = safe_ones.sample(frac = percentage, random_state = 33)#TODO
print(len(risky_loans))
print(len(safe_loans))
# # combine two kinds of loans
data_set = pd.concat([risky_loans, safe_loans], axis = 0)#TODO

Now, let's verify that the resulting percentage of safe and risky loans are each nearly 50%.

In [ ]:
data_set[target].value_counts(normalize=True)

## Preprocessing your features

In [ ]:
def dummies(data, columns=['pclass','name_title','embarked', 'sex']):
    for col in columns:
        data[col] = data[col].apply(lambda x: str(x))
        new_cols = [col + '_' + i for i in data[col].unique()]
        data = pd.concat([data, pd.get_dummies(data[col], prefix=col)[new_cols]], axis=1)
        del data[col]
    return data

In [ ]:
#grade, home_ownership, target
cols = ['grade', 'term','home_ownership', 'emp_length']
data_set = dummies(data_set, columns=cols)
data_set.head()

## 将数据分成训练集和测试集

重要的事情说三遍!!  

**把你的爪子从TEST DATA上拿开!!**   
**把你的爪子从TEST DATA上拿开!!**  
**把你的爪子从TEST DATA上拿开!!**  


In [ ]:
train_data, test_data = train_test_split(data_set, test_size = 0.2, random_state = 0)#TODO
trainX, trainY = train_data[train_data.columns[1:]], pd.DataFrame(train_data[target])#TODO
testX, testY = test_data[test_data.columns[1:]], pd.DataFrame(test_data[target])

In [ ]:
train_data.safe_loans

## 建自己的决策树!  

任务：  
1 实现根据error来选择最佳划分特征的函数best_split()  
2 实现根据entropy来选择最佳特征的函数best_split_entropy()  
3 实现树节点的类TreeNode  
4 实现模型的类MyDecisionTree  

#### 任务1， 实现根据error来选择最佳划分特征的函数best_split()  
约定树的左边对应target == 0， 树的右边对应target == 1

In [ ]:
def count_errors(labels_in_node):
    if len(labels_in_node) == 0:
        return 0
    
    positive_ones = labels_in_node.apply(lambda x: x== 1).sum()#TODO
    negative_ones = labels_in_node.apply(lambda x: x== -1).sum() #TODO
    
    return min(positive_ones, negative_ones) # TODO


def best_split(data, features, target):
    # return the best feature
    best_feature = None
    best_error = 2.0 
    num_data_points = float(len(data))  

    for feature in features:
        
        # 左分支对应当前特征为0的数据点
        left_split = data[data[feature] == 0]# TODO
        
        # 右分支对应当前特征为1的数据点
        right_split = data[data[feature] == 1]#TODO
        
        # 计算左边分支里犯了多少错
        left_misses = count_errors(left_split[target])#TODO            

        # 计算右边分支里犯了多少错
        right_misses = count_errors(right_split[target]) #TODO
            
        # 计算当前划分之后的分类犯错率
        error = (right_misses + left_misses) *1.0 /len(num_data_points)#TODO

        # 更新应选特征和错误率，注意错误越低说明该特征越好
        if error < best_error:
            best_error = error#TODO
            best_feature = feature#TODO
    return best_feature

#### 任务2， 实现根据entropy来选择最佳特征的函数best_split_entropy()  


In [ ]:
def entropy(labels_in_node):
    n = len(labels_in_node)
    s1 = (labels_in_node==1).sum()
    if s1 == 0 or s1 == n:
        return 0
    p1 = float(s1)/n
    
    p0 = 1- p1
    return -p0*np.log2(p0)- p1*np.log2(p1)

In [ ]:
def best_split_entropy(data, features, target):
    
    best_feature = None
    best_info_gain = float('-inf') 
    num_data_points = float(len(data))
    # 计算划分之前数据集的整体熵值
    entropy_original = entropy(data[target])#TODO

    for feature in features:
        
        # 左分支对应当前特征为0的数据点
        left_split = data[data[feature] == 0]#TODO
        
        # 右分支对应当前特征为1的数据点
        right_split = data[data[feature] == 1]#TODO 
        
        # 计算左边分支的熵值
        left_entropy = entropy(left_split[target])#TODO           

        # 计算右边分支的熵值
        right_entropy = entropy(right_split[target])#TODO
            
        # 计算左边分支与右分支熵值的加权和（数据集划分后的熵值）
        entropy_split = len(left_split)/ num_data_points * left_entropy + len(right_split)/num_data_points*right_entropy#TODO
        
        # 计算划分前与划分后的熵值差得到信息增益
        info_gain = entropy_original - entropy_split #TODO

        # 更新最佳特征和对应的信息增益的值
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_feature = feature
    return best_feature
    

In [ ]:
best_split_entropy(train_data, trainX.columns, target)

#### 任务3，实现树节点的类TreeNode，每个树节点应该包含如下信息:  

   3.1 is_leaf: True/False  表示当前节点是否为叶子节点  
   
   3.2 prediction: 当前节点做全民公投的预测结果
   
   3.3 left: 左子树  
   
   3.4 right: 右子树 
   
   3.5 split_feature: 当前节点用来划分数据时所采用的特征

In [ ]:
class TreeNode:
    def __init__(self, is_leaf, prediction, split_feature):
    # TODO
        self.is_leaf = is_leaf
        self.prediciton = prediction
        self.split_feature = split_feature
        self.left = None
        self.right = None       

#### 任务4，实现模型的类MyDecisionTree， 实现如下主要函数  
  
  
   4.1 fit(): 模型在训练集上的学习  
   
   4.2 predict(): 模型在数据集上的预测
   
   4.3 score(): 模型在测试集上的得分   
   
   
   
   
   为了实现4.1 - 4.3的方法， 需要实现如下辅助函数  
   4.4 create_tree(): 创建一棵树  
   
   4.5 create_leaf(): 创建叶子节点  
   
   4.6 predict_single_data(): 模型预测单个数据  
   
   4.7 count_leaves(): 统计模型的叶子数

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
class MyDecisionTree(BaseEstimator):
    
    def __init__(self, max_depth, min_error):
        self.max_depth = max_depth
        self.min_error = min_error
    
    def fit(self, X, Y, data_weights = None):
        
        data_set = pd.concat([X, Y], axis=1)
        features = X.columns#TODO
        target = Y.columns[0]
        # need to get teh depth
        self.root_node = self.create_tree(data_set, features, target, current_depth=0, max_depth = self.max_depth, min_error= self.min_error)# TODO
        
        
    def predict(self, X):
        prediction = X.apply(lambda row: self.predict_single_data(self.root_node, row), axis=1)
        return prediction
        
        
    def score(self, testX, testY):
        target = testY.columns[0]# TODO
        result = self.predict(testX)
        return accuracy_score(testY[target], result)
    
    
    def create_tree(self, data, features, target, current_depth = 0, max_depth = 10, min_error=0):
        """
        探索三种不同的终止划分数据集的条件  
  
        termination 1, 当错误率降到min_error以下, 终止划分并返回叶子节点  
        termination 2, 当特征都用完了, 终止划分并返回叶子节点  
        termination 3, 当树的深度等于最大max_depth时, 终止划分并返回叶子节点
        """        
    
        # 拷贝以下可用特征
        remaining_features = features[:]#TODO
        target_values = data[target]# TODO

        # termination 1
        if count_errors(target_values) <= min_error:
            print("Termination 1 reached.")     
            return self.create_leaf(target_values)# TODO

        # termination 2
        if len(remaining_features) == 0:
            print("Termination 2 reached.")    
            return self.create_leaf(target_values)# TODO    

        # termination 3
        if current_depth >= max_depth: 
            print("Termination 3 reached.")
            return self.create_leaf(target_values)# TODO
        
        # 选出最佳当前划分特征
        #split_feature = # TODO   #根据正确率划分
        split_feature = best_split_entropy(data, features, target)# TODO  # 根据信息增益来划分

        # 选出最佳特征后，该特征为0的数据分到左边，该特征为1的数据分到右边
        left_split = data[data[split_feature] == 0]# TODO
        right_split = data[data[split_feature] == 1]# TODO

        # 剔除已经用过的特征
        remaining_features = remaining_features.drop(split_feature)# TODO
        print("Split on feature %s. (%s, %s)" % (split_feature, str(len(left_split)), str(len(right_split))))

        # 如果当前数据全部划分到了一边，直接创建叶子节点返回即可
        if len(left_split) == len(data):
            print("Perfect split!")
            return self.create_leaf(left_split[target])
        if len(right_split) == len(data):
            print("Perfect split!")
            return self.create_leaf(right_split[target])

        # 递归上面的步骤
        #create_tree(data, features, target, current_depth = 0, max_depth = 10, min_error=0)
        left_tree = self.create_tree(left_split, remaining_features, target, current_depth+1, max_depth, min_error)# TODO     
        right_tree = self.create_tree(right_split, remaining_features, target, current_depth+1, max_depth, min_error)# TODO

        #生成当前的树节点
        result_node = TreeNode(False, None, split_feature)
        result_node.left = left_tree
        result_node.right = right_tree
        return result_node
    
    def create_leaf(self, target_values):
        # 用于创建叶子的函数
        
        # 初始化一个树节点
        leaf = TreeNode(True, None, None)# TODO

        # 统计当前数据集里标签为+1和-1的个数，较大的那个即为当前节点的预测结果
                ### ??? what is the datatype of target_values
        num_positive_ones = len(target_values[target_values == +1])
        num_negative_ones = len(target_values[target_values == -1])
        '''
        ### important, don't make such mistake again, apply lambda doesnt drop the unmatched elements from the list
        # num_positive_ones = len(target_values.apply(lambda x: x== +1))#TODO
        # num_negative_ones = len(target_values.apply(lambda x: x== -1))# TODO
        '''

        if num_positive_ones > num_negative_ones:
            leaf.prediction = 1
        else:
            leaf.prediction = -1

        # 返回叶子        
        return leaf 
    
    
    
    def predict_single_data(self, tree, x, annotate = False):   
        #Q1: what is x, what is the datatype
        # If tree is already a leaf, return the prediction
        if tree.is_leaf:
            if annotate: 
                print("leaf node, predicting %s" % tree.prediction)
            return tree.prediction# TODO 
        else:
            # check what the featuer sets, assumming x is the feature set
            split_feature_value = x[tree.split_feature]# TODO

            if annotate: 
                print("Split on %s = %s" % (tree.split_feature, split_feature_value))
            if split_feature_value == 0:
                #如果数据在该特征上的值为0，交给左子树来预测
                return self.predict_single_data(tree.left, x, None)# TODO
            else:
                #如果数据在该特征上的值为0，交给右子树来预测
                return self.predict_single_data(tree.right, x, None)# TODO    
    
    def count_leaves(self):
        return self.count_leaves_helper(self.root_node)
    
    def count_leaves_helper(self, tree):
        # TODO
        if tree.is_leaf: 
            return 1
        return self.count_leaves_helper(tree.left) + self.count_leaves_helper(tree.right)


In [ ]:
m = MyDecisionTree(max_depth = 10, min_error = 1e-15)

In [ ]:
m.fit(trainX, trainY)

In [ ]:
m.score(testX, testY)

### 决策树复杂度的探讨

In [ ]:
m.count_leaves()

#### 探索不同树深度对决策树的影响  

1 max_depth = 3  
2 max_depth = 7  
3 max_depth = 15


In [ ]:
model_1 = MyDecisionTree(max_depth = 3, min_error = 1e-15)# TODO
model_2 = MyDecisionTree(max_depth = 7, min_error = 1e-15)# TODO
model_3 = MyDecisionTree(max_depth = 15, min_error = 1e-15)# TODO


In [ ]:
model_1.fit(trainX, trainY)
model_2.fit(trainX, trainY)
model_3.fit(trainX, trainY)

In [ ]:
print("model_1 training accuracy :", model_1.score(trainX, trainY))
print("model_2 training accuracy :", model_2.score(trainX, trainY))
print("model_3 training accuracy :", model_3.score(trainX, trainY))

In [ ]:
print("model_1 testing accuracy :", model_1.score(testX, testY))
print("model_2 testing accuracy :", model_2.score(testX, testY))
print("model_3 testing accuracy :", model_3.score(testX, testY))

In [ ]:
print("model_1 complexity is: ", model_1.count_leaves())
print("model_2 complexity is: ", model_2.count_leaves())
print("model_3 complexity is: ", model_3.count_leaves())